# Notebook de Exemplos: Interpolação de Pontos na Reta Real

Este notebook demonstra o uso das seguintes funções de interpolação:

1.  `lin_interp`: Realiza a interpolação linear por partes entre os pontos.

2.  `poly_interp`: Calcula o polinômio interpolador (Interpolação Polinomial).

3.  `hermite_interp`: Calcula o polinômio interpolador de Hermite (usando valores e derivadas).

Todas as funções também demonstram a representação gráfica dos pontos e do polinômio/função interpoladora.